In [1]:
# export LD_LIBRARY_PATH=/home/shenwanxiang/anaconda3/envs/IRnet_env/lib:$LD_LIBRARY_PATH
# nohup /home/shenwanxiang/anaconda3/envs/IRnet_env/bin/python RunTransferLearning.py > RunTransferLearning.log 2>&1 &

import os
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/home/shenwanxiang/anaconda3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
tf.test.is_gpu_available()
tf.config.run_functions_eagerly(True)
tf.__version__

2025-08-15 02:01:39.638268: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-15 02:01:39.706176: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-15 02:01:40.057458: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/shenwanxiang/anaconda3/lib:
2025-08-15 02:01:40.057492: W tensorflow/compiler/xla/stream_exe

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2025-08-15 02:01:40.462622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-15 02:01:40.472681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-15 02:01:40.488486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-15 02:01:40.488573: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at

'2.11.0'

In [2]:
tf.test.is_gpu_available()

2025-08-15 02:01:40.774202: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-15 02:01:40.774323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-15 02:01:40.774382: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-15 02:01:40.774457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-15 02:01:40.774513: I tensorflow/compiler/xla/stream_executo

True

_device.cc:1613] Created device /device:GPU:0 with 9998 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
import gc

import sys
sys.path.append('../codeocean/environment/clinical_transformer/')
from xai.models import Trainer
from xai.models import SurvivalTransformer
from xai.models import OptimizedSurvivalDataGenerator as SurvivalDataGenerator
from xai.losses.survival import cIndex_SigmoidApprox as cindex_loss
from xai.metrics.survival import sigmoid_concordance as cindex_metric
from xai.models.explainer import TransformerSurvivalEvaluator
from xai.models.explainer import survival_attention_scores
from xai.models import load_transformer

import pandas as pd
from samecode.random import set_seed

import numpy as np 
import seaborn as sns
from matplotlib import pyplot as plt
from samecode.survival.plot import KMPlot
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index as lfcindex
import seaborn as sns
from shap.plots import colors
from glob import glob
import os
colors = [colors.blue_rgb, colors.red_rgb]

## PARAMETERS
max_features_percentile=100
test_size=0.1 # fraction of samples used for validation
repetitions=2  # number replicates (training / validation) random splits to evaluate variability.
mode='survival'
learning_rate=0.0001
epochs=3
verbose=1

embedding_size = 128
num_heads = 2
num_layers = 8


SEED = [24, 42, 64]


def get_best_epoch(train_data, features, pretrained_path, 
                   work_dir = './results/model/cohort'):
    
    outdir = work_dir
    os.system('rm -r %s' % outdir)

    set_seed(0)

    trainer = Trainer(
        from_pretrained=pretrained_path,
        out_dir = outdir,
        max_features_percentile=max_features_percentile,
        test_size=test_size,
        mode=mode,
        model=SurvivalTransformer, 
        dataloader=SurvivalDataGenerator,
        loss=cindex_loss,
        metrics=[cindex_metric]
    )

    trainer.setup_data(
        train_data, 
        discrete_features = [],
        continuous_features = features,
        target=['time', 'event']
    )

    trainer.setup_model(
        learning_rate=learning_rate,
        embedding_size=embedding_size,
        num_heads=num_heads,
        num_layers=num_layers,
        batch_size_max=True,
        save_best_only=False
    )

    trainer.fit(repetitions=repetitions, 
                epochs=epochs, 
                verbose=verbose, 
                seed=0)
    
    del trainer; tf.keras.backend.clear_session(); gc.collect()
    best_es = []
    for i in range(repetitions):
        dfh = pd.read_csv(f"{outdir}/model.E002000.h5/fold-{i}_id-{i}/history.csv", index_col=0)
        #fig, ax=plt.subplots(figsize=(5,4))
        #dfh[['loss', 'val_loss']].plot(ax=ax)
        #ax.set_ylabel('C-index loss')
        best_es.append(dfh.val_sigmoid_concordance)
    best_epoch = pd.concat(best_es, axis=1).mean(axis=1).idxmax() + 1

    return best_epoch


def train(train_data, features, pretrained_path, best_epoch, 
                  work_dir = './results/TransferLearningSurvival/'
                  ):

    all_dirs = []
    for seed in SEED:
        set_seed(seed)
        
        outdir = f'{work_dir}/full_{seed}'
        os.system('rm -r %s' % outdir)
        
        trainer = Trainer(
            from_pretrained=pretrained_path,
            out_dir = outdir,
            max_features_percentile=max_features_percentile,
            test_size=0.,
            mode=mode,
            model=SurvivalTransformer, 
            dataloader=SurvivalDataGenerator,
            loss=cindex_loss,
            metrics=[cindex_metric]
        )

        trainer.setup_data(
            train_data, 
            discrete_features = [],
            continuous_features = features,
            target=['time', 'event']
        )

        trainer.setup_model(
            learning_rate=learning_rate,
            embedding_size=embedding_size,
            num_heads=num_heads,
            num_layers=num_layers,
            batch_size_max=True,
            save_best_only=False
        )

        trainer.fit(repetitions=1, 
                    epochs=best_epoch, 
                    verbose=verbose, 
                    seed=seed)
        
        del trainer; tf.keras.backend.clear_session(); gc.collect()
        
        all_dirs.append(outdir)
        
    return all_dirs


def predict(train_data, test_data, features, best_epoch, model_dirs):

    predicted_paths = []
    for path_dir in model_dirs:
        sample_id = 'Index'
        run = 'fold-0_id-0'
        path = f'{path_dir}/model.E002000.h5'
        
        repeat = path_dir.split('/')[-1]
        cohort = path_dir.split('/')[-2]  
        model = path_dir.split('/')[-3]        
        
        trainer = load_transformer(path, run, epoch=best_epoch)
        transformed_data = trainer.data_converter.transform(train_data).reset_index(drop=True)
        set_seed(0)
        evaluator = TransformerSurvivalEvaluator(model=trainer)
        train_data, outputs, features, patient_ids, iters, attention_scores = survival_attention_scores(
            transformed_data, evaluator, iterations=1, sample_id=sample_id
        )
        train_cindex = lfcindex(train_data['time'], train_data['β'], train_data['event'])
        transformed_test_data = trainer.data_converter.transform(test_data).reset_index(drop=True)
        evaluator = TransformerSurvivalEvaluator(model=trainer)

        test_data, outputs, features, patient_ids, iters, attention_scores = survival_attention_scores(
            transformed_test_data, evaluator, iterations=10, sample_id=sample_id
        )
        test_cindex = lfcindex(test_data['time'], test_data['β'], test_data['event'])

        del trainer; tf.keras.backend.clear_session(); gc.collect()
        median_cutoff = np.quantile(train_data.β, 0.75)


        test_data['population'] = (test_data.β >= median_cutoff).replace([False, True], ['Low Score', 'High Score'])
        train_data['population'] = (train_data.β >= median_cutoff).replace([False, True], ['Low Score', 'High Score'])

        fig, axs = plt.subplots(ncols=2, nrows=1, figsize = (12,5))
        KMPlot(train_data, time='time', event='event', label=['population']).plot(ax=axs[0], 
                                                                              colors = colors, 
                                                                              title='Training set',
                                                                              ci_show=True,ci_alpha=0.10,

                                                                             )
        KMPlot(test_data, time='time', event='event', label=['population']).plot(ax=axs[1],
                                                                                 colors = colors, ci_show=True,
                                                                                 ci_alpha=0.10,
                                                                                 title=f'Testing set ({cohort})')

        
        test_data['test_cohort'] = cohort
        test_data['model'] = model
        test_data['repeat'] = repeat
        
        train_data['test_cohort'] = train_data['cohort']
        train_data['model'] = model
        train_data['repeat'] = repeat
        
        test_data.to_csv(os.path.join(path, 'test_data.csv'), index = False)
        train_data.to_csv(os.path.join(path, 'train_data.csv'), index = False)
        fig.savefig(os.path.join(path, 'train_test_km.svg'))
        predicted_paths.append(os.path.join(path, 'test_data.csv'))
    
    return predicted_paths



y = pd.read_csv('../data/ITRP_clinical.csv', index_col=0)
y = y[(~y.OS_Months.isna()) & (~y.OS_Event.isna())]
y['time'] = y['OS_Months']
y['event'] = y['OS_Event']
y = y[['time', 'event','cohort', 'ICI_target', 'ICI','cancer_type','response_label', 'TMB']]
cohort_list = y.cohort.unique()


Model_NAMES = ['VanillaClinicalTransformer',
              'ssGSEA43ClinicalTransformer',
              'COMPASSClinicalTransformer',]

Model_PATHS = ['01_run_Feg29_TME',
              '02_run_ssGSEA43_TME',
              '03_run_Com44_TME',]

X_PATHS = ['ITRP_ssGSEA_Fegs29',
          'ITRP_ssGSEA_concept43',
          'ITRP_COMPASS_concept44']


save_dir = './ResultsTest'
os.makedirs(save_dir, exist_ok=True)

for model_name, pt_path, X_file_path in zip(Model_NAMES,Model_PATHS, X_PATHS):

    pretrained_path = f'./{pt_path}/FoundationModel/fold-0_id-0/model.E002000.h5'
    X = pd.read_csv(f'../data/{X_file_path}.csv', index_col=0)
    dataset_full = y.join(X)
    features = X.columns.tolist()

    for test_cohort in cohort_list:
        train_data = dataset_full[dataset_full.cohort != test_cohort].reset_index()
        test_data = dataset_full[dataset_full.cohort == test_cohort].reset_index()
        
        best_epoch = get_best_epoch(train_data, features, pretrained_path, 
                           work_dir = f'{save_dir}/{model_name}/{test_cohort}')
        
        model_dirs = train(train_data, features, pretrained_path, best_epoch, 
                          work_dir = f'{save_dir}/{model_name}/{test_cohort}'
                          )
        
        pred_dirs = predict(train_data.copy(), test_data.copy(), 
                            features, best_epoch, model_dirs)

        tf.keras.backend.clear_session(); gc.collect()

/home/shenwanxiang/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
sh: /home/shenwanxiang/anaconda3/lib/libtinfo.so.6: no version information available (required by sh)
rm: cannot remove './ResultsTest/VanillaClinicalTransformer/IMVigor210': No such file or directory
2025-08-15 02:01:52.617700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-15 02:01:52.617860: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-08-15 02:01:52.617924: I tensorflow/compiler/xla/stream_executor/cuda

Epoch 1/3


Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
2025-08-15 02:01:54.034014: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


WARNING	2025-08-15 02:01:54,284	AutoGraph could not transform <function cIndex_SigmoidApprox at 0x7fb872c2b710> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Epoch 1/3
1/1 [==============================] - 1s 565ms/step - loss: 0.4707 - sigmoid_concordance: 0.5590 - val_loss: 0.6005 - val_sigmoid_concordance: 0.4118
Epoch 2/3
1/1 [==============================] - 0s 349ms/step - loss: 0.4473 - sigmoid_concordance: 0.5841 - val_loss: 0.6254 - val_sigmoid_concordance: 0.4412
Epoch 3/3
1/1 [==============================] - 0s 357ms/step - loss: 0.4245 - sigmoid_concordance: 0.6015 - val_loss: 0.6392 - val_sigmoid_concordance: 0.4118


 63%|██████████████████████████████████████████████████▍                             | 510/809 [00:01<00:01, 285.11it/s]


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fb86bfc84d0> (for post_execute):


KeyboardInterrupt: 